In [1]:
import os 
from Bio import SeqIO
import pickle
import numpy as np

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier

import math

import pandas as pd

from DNA_Encoding import DNA_Encoding

# from sklearn.model_selection import RandomizedSearchCV

In [2]:
# labels

In [3]:
# n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# max_features = ['auto', 'sqrt']
# max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
# max_depth.append(None)
# min_samples_split = [2, 5, 10]
# min_samples_leaf = [1, 2, 4]
# bootstrap = [True, False]

# random_grid = {'n_estimators': n_estimators,
#                'max_features': max_features,
#                'max_depth': max_depth,
#                'min_samples_split': min_samples_split,
#                'min_samples_leaf': min_samples_leaf,
#                'bootstrap': bootstrap}

# model = RandomForestClassifier(n_estimators = 100, random_state = 42, criterion = "gini")

# model_random = RandomizedSearchCV(estimator = model, param_distributions = random_grid, n_iter = 100, cv = 10, verbose=2, random_state=42, n_jobs = -1)

# # model_random.fit(fold["X_train"], fold["y_train"].reshape(fold["y_train"].shape[0]))
# model_random.fit(features, labels)

In [4]:
##################################################################################
##### Define all parameters for model tuning
##################################################################################

n_fold = 10
shuffle = False
seed = None

# expName = "embedReads_setting1_DNAGlobalEncoding_backup"
expName = "embedReads_setting1_DNAGlobalEncoding"

dataset_path = "CORENup-Datasets\\Datasets"
setting = "Setting1"

outPath = "Generated"
foldName = "folds.pickle"

modelName = "RandomForest"

dataset_setting_path = os.path.join(dataset_path, setting)

obj_DNA_Encoding = DNA_Encoding

In [5]:
##################################################################################
##### define evaluator functions
##################################################################################

## Build the K-fold from dataset
def build_kfold(features, labels, k=10, shuffle=False, seed=None):
    
    skf = StratifiedKFold(n_splits=k, shuffle=shuffle, random_state=seed)
    kfoldList = []
    for train_index, test_index in skf.split(features, labels):
        X_train, X_test = features[train_index], features[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        kfoldList.append({
            "X_train": X_train,
            "X_test": X_test,
            "y_train":y_train,
            "y_test":y_test
        })
    return kfoldList

# def pred2label(y_pred):
#     y_pred = np.round(np.clip(y_pred, 0, 1))
#     return y_pred

In [6]:
##################################################################################
##### For each input file, train model and generate different outputs in a structured folder
##################################################################################

## create the evaluation data structure for all iterations
evaluations = {
    "Model" : [],
    "Kernel_Length" : [],
    "Dataset" : [],
    "Fold" : [],
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

for root, dirs, files in os.walk(dataset_setting_path):
    for file in files:
        
        current_dataset_variety = root.split("\\")[len(root.split("\\"))-1]
        
        ##################################################################################
        ##### read the current file
        ##################################################################################

        openFile = open(os.path.join(root, file))
        fastaSequences = SeqIO.parse(openFile, "fasta")

        ##################################################################################
        ##### extract data from the current fasta file
        ##################################################################################
        
        nucleosomal_List = []
        linker_List = []
        nucleosomal_encoded_List = []
        linker_encoded_List = []

        for fasta in fastaSequences: 
            name, sequence = fasta.id, str(fasta.seq)
            if "nucleosomal" in name:
                nucleosomal_List.append(sequence)
                aus_seq = DNA_Encoding.GlobalEncoding(self = obj_DNA_Encoding, seq = sequence)
                if(len(aus_seq) != 0):
                    nucleosomal_encoded_List.append(aus_seq)
            else:
                linker_List.append(sequence)
                aus_seq = DNA_Encoding.GlobalEncoding(self = obj_DNA_Encoding, seq = sequence)
                if(len(aus_seq) != 0):
                    linker_encoded_List.append(aus_seq)

        print("\n======================================================================")
        print("\nFile: "+os.path.join(root, file))
        print("Nucleosomi: "+str(len(nucleosomal_encoded_List)))
        print("Linker: "+str(len(linker_encoded_List)))

        ##################################################################################
        ##### Generate Folds from dataset, and store to file
        ##################################################################################

        ## create the features and labels datasets for the training
        # input_size = (len(nucleosomal_encoded_List[1]), 4)
        labels = np.concatenate((np.ones((len(nucleosomal_encoded_List), 1), dtype=np.float32), np.zeros((len(linker_encoded_List), 1), dtype=np.float32)), axis=0)
        features = np.concatenate((nucleosomal_encoded_List,linker_encoded_List), 0)

        ## Generate the k-fold dataset
        folds = build_kfold(features, labels, k=n_fold, shuffle=shuffle, seed=seed)

        ## Write the k-fold dataset to file
        foldPath = os.path.join(outPath, expName, current_dataset_variety, "{}fold".format(n_fold))

        ##### ADDITIONAL CHANGES - USE PREVIOUS GENERATED FOLDS IF AVAILABLE

        if(os.path.isfile(os.path.join(foldPath, foldName))):
            folds = pickle.load(open(os.path.join(foldPath, foldName), "rb"))
        else:
            if(not os.path.isdir(foldPath)):
                os.makedirs(foldPath)
            pickle.dump(folds, open(os.path.join(foldPath, foldName), "wb"))

        ##################################################################################
        ##### TRAIN and PREDICT for every Fold, using Random Forest
        ##################################################################################

        ## Create and set directory to save model
        modelPath = os.path.join(outPath, expName, current_dataset_variety, "{}fold".format(n_fold), "models", modelName)
        if(not os.path.isdir(modelPath)):
            os.makedirs(modelPath)

        ## fold counter
        i = 0

        for fold in folds:

            print("\nTrain/Test model "+modelName+" on Fold #"+str(i)+".")

            ## Generate model Random Forest
            model = RandomForestClassifier(n_estimators = 100, random_state = 42, oob_score = True, criterion = "gini")

            model.fit(X = fold["X_train"], 
                      y = fold["y_train"].reshape(fold["y_train"].shape[0]))
            
            pickle.dump(model, open(os.path.join(modelPath, "{}_bestModel-fold{}.pkl".format(modelName, i)), 'wb'))
            
            ##################################################################################
            ##### Prediction and metrics for TRAIN dataset
            ##################################################################################

            y_pred = model.predict(fold["X_train"])
            #label_pred = pred2label(y_pred)
            label_pred = y_pred
            # Compute precision, recall, sensitivity, specifity, mcc
            acc = accuracy_score(fold["y_train"], label_pred)
            prec = precision_score(fold["y_train"],label_pred)

            conf = confusion_matrix(fold["y_train"], label_pred)
            if(conf[0][0]+conf[1][0]):
                sens = float(conf[0][0])/float(conf[0][0]+conf[1][0])
            else:
                sens = 0.0
            if(conf[1][1]+conf[0][1]):
                spec = float(conf[1][1])/float(conf[1][1]+conf[0][1])
            else:
                spec = 0.0
            if((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0])):
                mcc = (float(conf[0][0])*float(conf[1][1]) - float(conf[1][0])*float(conf[0][1]))/math.sqrt((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0]))
            else:
                mcc= 0.0
            fpr, tpr, thresholds = roc_curve(fold["y_train"], y_pred)
            auc = roc_auc_score(fold["y_train"], y_pred)

            evaluations["Model"].append(modelName)
            evaluations["Kernel_Length"].append("")
            evaluations["Dataset"].append(current_dataset_variety)
            evaluations["Fold"].append(i)
            evaluations["Train_Test"].append("Train")
            evaluations["Accuracy"].append(acc)
            evaluations["Precision"].append(prec)
            evaluations["TPR"].append(tpr)
            evaluations["FPR"].append(fpr)
            evaluations["TPR_FPR_Thresholds"].append(thresholds)
            evaluations["AUC"].append(auc)
            evaluations["Sensitivity"].append(sens)
            evaluations["Specificity"].append(spec)
            evaluations["MCC"].append(mcc)

            ##################################################################################
            ##### Prediction and metrics for TEST dataset
            ##################################################################################

            y_pred = model.predict(fold["X_test"])
            # label_pred = pred2label(y_pred)
            label_pred = y_pred
            # Compute precision, recall, sensitivity, specifity, mcc
            acc = accuracy_score(fold["y_test"], label_pred)
            prec = precision_score(fold["y_test"],label_pred)

            conf = confusion_matrix(fold["y_test"], label_pred)
            if(conf[0][0]+conf[1][0]):
                sens = float(conf[0][0])/float(conf[0][0]+conf[1][0])
            else:
                sens = 0.0
            if(conf[1][1]+conf[0][1]):
                spec = float(conf[1][1])/float(conf[1][1]+conf[0][1])
            else:
                spec = 0.0
            if((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0])):
                mcc = (float(conf[0][0])*float(conf[1][1]) - float(conf[1][0])*float(conf[0][1]))/math.sqrt((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0]))
            else:
                mcc= 0.0
            fpr, tpr, thresholds = roc_curve(fold["y_test"], y_pred)
            auc = roc_auc_score(fold["y_test"], y_pred)

            evaluations["Model"].append(modelName)
            evaluations["Kernel_Length"].append("")
            evaluations["Dataset"].append(current_dataset_variety)
            evaluations["Fold"].append(i)
            evaluations["Train_Test"].append("Test")
            evaluations["Accuracy"].append(acc)
            evaluations["Precision"].append(prec)
            evaluations["TPR"].append(tpr)
            evaluations["FPR"].append(fpr)
            evaluations["TPR_FPR_Thresholds"].append(thresholds)
            evaluations["AUC"].append(auc)
            evaluations["Sensitivity"].append(sens)
            evaluations["Specificity"].append(spec)
            evaluations["MCC"].append(mcc)

            i = i+1
            del model

##################################################################################
##### Dump evaluations to a file
##################################################################################

evalPath = os.path.join(outPath, expName, "_Evaluation_All_Datasets")
if(not os.path.isdir(evalPath)):
    os.makedirs(evalPath)

pickle.dump(evaluations,
            open(os.path.join(evalPath, "{}fold_evaluations_{}.pickle".format(n_fold, modelName)), "wb"))




File: CORENup-Datasets\Datasets\Setting1\Drosophila\nucleosomes_vs_linkers_melanogaster.fas
Nucleosomi: 2900
Linker: 2850

Train/Test model RandomForest on Fold #0.

Train/Test model RandomForest on Fold #1.

Train/Test model RandomForest on Fold #2.

Train/Test model RandomForest on Fold #3.

Train/Test model RandomForest on Fold #4.

Train/Test model RandomForest on Fold #5.

Train/Test model RandomForest on Fold #6.

Train/Test model RandomForest on Fold #7.

Train/Test model RandomForest on Fold #8.

Train/Test model RandomForest on Fold #9.


File: CORENup-Datasets\Datasets\Setting1\Elegans\nucleosomes_vs_linkers_elegans.fas
Nucleosomi: 2567
Linker: 2608

Train/Test model RandomForest on Fold #0.

Train/Test model RandomForest on Fold #1.

Train/Test model RandomForest on Fold #2.

Train/Test model RandomForest on Fold #3.

Train/Test model RandomForest on Fold #4.

Train/Test model RandomForest on Fold #5.

Train/Test model RandomForest on Fold #6.

Train/Test model RandomFores

KeyboardInterrupt: 

## Visualization of Evaluation

In [ ]:
##################################################################################
##### Add import statement here, to make this next part of code standalone executable
##################################################################################

import os
import pickle
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter, FormatStrFormatter
import numpy as np
import pandas as pd
import math


In [ ]:
# ##################################################################################
# ##### Parameters used only in this section
# ##################################################################################

# n_fold = 10
# expName = "embedReads_setting1_DNAGlobalEncoding"
# outPath = "Generated"

# modelNames = ["RandomForest"]

In [ ]:
##################################################################################
##### Load file and convert to dataframe for easy manipulation
##################################################################################

evalPath = os.path.join(outPath, expName, "_Evaluation_All_Datasets")
if(not os.path.isdir(evalPath)):
    os.makedirs(evalPath)

evaluations = pickle.load(open(os.path.join(evalPath, "{}fold_evaluations_{}.pickle".format(n_fold, modelName.format("all"))), "rb"))

evaluations_df = pd.DataFrame.from_dict(evaluations)

In [ ]:
##################################################################################
##### Group dataset (mean of metrics) by [Dataset, Model, Train_Test] combinations
##################################################################################

evaluations_df_grouped = evaluations_df.groupby(["Dataset", 
                                                 "Model", 
                                                 "Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

RF_Train = evaluations_df_grouped[np.in1d(evaluations_df_grouped.index.get_level_values(2), ['Train'])]
RF_Test = evaluations_df_grouped[np.in1d(evaluations_df_grouped.index.get_level_values(2), ['Test'])]

In [ ]:
##################################################################################
##### Decide on metric to visualize
##################################################################################

print("Metrics Available : ")
print(list(evaluations_df_grouped.columns))

#### Select a metric to plot below:

In [ ]:
metric_to_plot = "Accuracy"

In [ ]:
##################################################################################
##### Visualize with a multiple Bar chart
##################################################################################

x = np.arange(len(RF_Train[metric_to_plot]))
width = 0.15

fig, ax = plt.subplots(figsize=(17,6))
rects1 = ax.bar(x - (1.5*(width/2)), round(RF_Train[metric_to_plot]*100, 3), width, label='Random Forest, Train')
rects3 = ax.bar(x + (1.5*(width/2)), round(RF_Test[metric_to_plot]*100, 3), width, label='Random Forest, Test')

## Custom y-axis tick labels
ax.set_ylabel(metric_to_plot)
ax.set_ylim([max((math.floor(min(evaluations_df_grouped[metric_to_plot])*10)-1)*10, 0), 
            max((math.ceil(max(evaluations_df_grouped[metric_to_plot])*10)+1)*10, 0)])
# ax.set_ylim([80, 105])

## Custom x-axis tick labels
ax.set_xticks(x)
ax.set_xticklabels(RF_Train.index.get_level_values(0))
# ax.set_xticklabels([m+" - "+str(n) for m,n in 
#                         zip(DLNN_CORENup_Train.index.get_level_values(0),DLNN_CORENup_Train.index.get_level_values(1))],
#                   rotation=30)
# ax.set_xticklabels([a + "_" + b.split("_")[1] for a, b in zip(RF_Train.index.get_level_values(0), RF_Train.index.get_level_values(1))],
#                   rotation=30)


ax.set_title(metric_to_plot+' by Dataset, Model, Train/Test')
ax.legend(loc='upper left')

def autolabel(rects):
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points", 
                    ha='center', va='bottom', rotation=90)

autolabel(rects1)
autolabel(rects3)

plt.show()

### Store all metrics' plots to file

In [ ]:
##################################################################################
##### Iteratively generate comparison plot using every metric
##################################################################################

for metric_to_plot in list(evaluations_df_grouped.columns):
    
    x = np.arange(len(RF_Train[metric_to_plot]))
    width = 0.15

    fig, ax = plt.subplots(figsize=(17,6))
    rects1 = ax.bar(x - (1.5*(width/2)), round(RF_Train[metric_to_plot]*100, 3), width, label='Random Forest, Train')
    rects3 = ax.bar(x + (1.5*(width/2)), round(RF_Test[metric_to_plot]*100, 3), width, label='Random Forest, Test')

    ## Custom y-axis tick labels
    ax.set_ylabel(metric_to_plot)
    ax.set_ylim([max((math.floor(min(evaluations_df_grouped[metric_to_plot])*10)-1)*10, 0), 
                max((math.ceil(max(evaluations_df_grouped[metric_to_plot])*10)+1)*10, 0)])
    # ax.set_ylim([80, 105])

    ## Custom x-axis tick labels
    ax.set_xticks(x)
    ax.set_xticklabels(RF_Train.index.get_level_values(0))
    # ax.set_xticklabels([m+" - "+str(n) for m,n in 
    #                         zip(DLNN_CORENup_Train.index.get_level_values(0),DLNN_CORENup_Train.index.get_level_values(1))],
    #                   rotation=30)
    # ax.set_xticklabels([a + "_" + b.split("_")[1] for a, b in zip(RF_Train.index.get_level_values(0), RF_Train.index.get_level_values(1))],
    #                   rotation=30)


    ax.set_title(metric_to_plot+' by Dataset, Model, Train/Test')
    ax.legend(loc='upper left')

    def autolabel(rects):
        for rect in rects:
            height = rect.get_height()
            ax.annotate('{}'.format(height),
                        xy=(rect.get_x() + rect.get_width() / 2, height),
                        xytext=(0, 3),  # 3 points vertical offset
                        textcoords="offset points", 
                        ha='center', va='bottom', rotation=90)

    autolabel(rects1)
    autolabel(rects3)
    
    plt.savefig(os.path.join(evalPath, "{}_{}_Comparison".format(metric_to_plot, modelName.format("all"))))
    plt.close()
    